In [1]:
from jenga.tasks.reviews import VideogameReviewsTask
from jenga.evaluation.schema_stresstest import SchemaStresstest

import tensorflow_data_validation as tfdv

import numpy as np
import pandas as pd

### Instantiate the video game reviews task with a randomly chosen seed

In [2]:
seed = np.random.randint(2**32 - 1)

task = VideogameReviewsTask(seed=seed)

/usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[True, False] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/usr/local/Caskroom/miniconda/base/envs/data-imputation-paper/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[True, False] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


### Create a tfdv schema by first auto-infering it from training data and then adjusting it

In [3]:
train_data_stats = tfdv.generate_statistics_from_dataframe(task.train_data)
schema = tfdv.infer_schema(statistics=train_data_stats)
review_date_feature = tfdv.get_feature(schema, 'review_date')
review_date_feature.distribution_constraints.min_domain_mass = 0.0

In [4]:
schema

feature {
  name: "marketplace"
  type: BYTES
  domain: "marketplace"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "customer_id"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "review_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_parent"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_title"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_category"
  type: BYT

### Train the baseline model for the task

In [5]:
model = task.fit_baseline_model(task.train_data, task.train_labels)

### Run a stress test for the schema and the model with 250 randomly chosen corruptions, and mark performance drops of 3% as failures

In [6]:
stress_test = SchemaStresstest()
results = stress_test.run(task, model, schema, num_corruptions=250, performance_threshold=.03)

MissingValues: {'column': 'customer_id', 'fraction': 0.44, 'sampling': 'MNAR', 'na_value': ''}
MissingValues: {'column': 'vine', 'fraction': 0.01, 'sampling': 'MCAR', 'na_value': ''}
MissingValues: {'column': 'product_title', 'fraction': 0.97, 'sampling': 'MCAR', 'na_value': ''}
BrokenCharacters: {'column': 'review_id', 'fraction': 0.19}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.49}
BrokenCharacters: {'column': 'product_id', 'fraction': 0.35}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.34}
MissingValues: {'column': 'marketplace', 'fraction': 0.52, 'sampling': 'MCAR', 'na_value': ''}
BrokenCharacters: {'column': 'verified_purchase', 'fraction': 0.24}
BrokenCharacters: {'column': 'product_id', 'fraction': 0.67}
BrokenCharacters: {'column': 'review_headline', 'fraction': 0.55}
BrokenCharacters: {'column': 'title_and_review_text', 'fraction': 0.6}
MissingValues: {'column': 'vine', 'fraction': 0.87, 'sampling': 'MNAR', 'na_value': ''}
BrokenCharacters: {'column

### Look at the dataframe containing the results

In [7]:
results

,corruption,status,anomalies,baseline_score,corrupted_score
0,"MissingValues: {'column': 'customer_id', 'frac...",FP,"{'customer_id': description: ""String values th...",0.79382,0.793820
1,"MissingValues: {'column': 'vine', 'fraction': ...",FP,"{'vine': description: ""Examples contain values...",0.79382,0.791407
2,"MissingValues: {'column': 'product_title', 'fr...",TN,{},0.79382,0.793820
3,"BrokenCharacters: {'column': 'review_id', 'fra...",TN,{},0.79382,0.793820
4,"BrokenCharacters: {'column': 'customer_id', 'f...",TN,{},0.79382,0.793820
...,...,...,...,...,...
245,BrokenCharacters: {'column': 'verified_purchas...,TN,{},0.79382,0.793820
246,"SwappedValues: {'column': 'review_headline', '...",TN,{},0.79382,0.793820
247,MissingValues: {'column': 'title_and_review_te...,FN,{},0.79382,0.735923
248,"BrokenCharacters: {'column': 'customer_id', 'f...",TN,{},0.79382,0.793820
